# Instructions for Code Execution
To execute this code and perform analyses, it's necessary to import the datagen_IID and datagen_non-iid files into the project folder.

Acknowledgements and References
I would like to express my sincere thanks to the author of the repository on GitHub, whose work was fundamental for the development of this project. The algorithm studied for generating non-iid data contributed significantly to the completion of this analysis.

Original Algorithm Author: Jeremy (https://github.com/jeremy313)  
Reference: [Non-iid Mnist Algorithm](https://github.com/jeremy313/non-iid-dataset-for-personalized-federated-learning/blob/master/dataset/mnist_noniid.py)
Modified by  
Modification Author: Ernesto Gurgel Valente Neto


# Dataset [MNIST]

### Functions [IDD DATASET]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy
from keras.src.utils.np_utils import to_categorical
from tensorflow.keras.datasets import cifar10
import time
from mnist_noniid import get_dataset_mnist_extr_noniid
from scipy.stats import norm
from scipy.stats import kstest
from scipy.stats import normaltest
from scipy import stats

def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def pegarDadosIDDMNIST():
  num_users_mnist = 1   #Quatidade de usuarios
  nclass_mnist = 9      #Quantidade de classes que cada usuario possui
  nsamples_mnist = 1    #cada shad divide o conjunto de dados de 60.000 em partes
  rate_unbalance_mnist = 0.0 #1 altamente desbalanceado e 0 seria balanceado
  train_dataset_mnist, test_dataset_mnist, user_groups_train_mnist, user_groups_test_mnist = get_dataset_mnist_extr_noniid(num_users_mnist, nclass_mnist, nsamples_mnist, rate_unbalance_mnist)
  print(train_dataset_mnist)
  all_images = [img[0].numpy().squeeze() for img in train_dataset_mnist]
  all_images_np = np.stack(all_images)
  #Removendo a mudança ao qual deixa os dados
  mean, std = 0.1307, 0.3081
  all_images_np = all_images_np * std + mean
  print(all_images_np.shape)
  all_labels = [lbl for _, lbl in train_dataset_mnist]
  # Plotting the histogram
  plt.hist(all_labels, bins=10, edgecolor='black', alpha=0.7)
  plt.title('Class Distribution Histogram')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()
  return train_dataset_mnist, test_dataset_mnist, user_groups_train_mnist, user_groups_test_mnist, all_images_np

def iEMM(train_X, base = 0, ftupla = 0):
  train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))   # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  print('Valor da Mediana: ', median)
  if base == 1:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #indices de entropia entropia baixa abaixo da mediana.
  elif base == 2:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] >= median])  #indices de entropia entropia baixa abaixo da mediana.
  if base != 0:
    train_X = np.array([train_X[i] for i in indices_filtrados]) # passa os indices selecionados
  if ftupla == 1:
    tuplasEntropia = [tuplasEntropia[i] for i in indices_filtrados]
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado):
  print("Tamanho do Dataset: ", len(tuplasEntropia))
  print("Tamanho do divido pela Mediana: ", len(train_X))
  print("Entropia das imagens (Não-Ordenado | Original): ", tuplasEntropia[:5], " ...")
  print("Entropia das imagens (Ordenado | Valor): ", entropiesLocal_ordenado[:5], " ...")

  valEnt = [(ento[1]) for ento in (tuplasEntropia)]
  soma = sum(valEnt)
  print("Entropia Total Acumulada: ", soma)
  alvos = [0.25 * soma, 0.5 * soma, 0.75 * soma, soma]
  grupos = []
  grupo_atual = []
  soma_acumulada = 0
  index_alvo = 0
  for num in valEnt:
      grupo_atual.append(num)
      soma_acumulada += num
      if soma_acumulada >= alvos[index_alvo]:
          grupos.append(grupo_atual)
          grupo_atual = []
          index_alvo += 1
          if index_alvo == 4:
              break
  if grupo_atual:
      grupos.append(grupo_atual)

  print("Tamanho do Conjunto Analisado do Grafico a seguir: ", len(valEnt))
  print("Tamanho do grupo 1 - 25% dos valores (Q1): ", len(grupos[0]))
  print("Tamanho do grupo 2 - 50% dos valores (Q2): ", len(grupos[1]))
  print("Tamanho do grupo 3 - 75% dos valores (Q3): ", len(grupos[2]))
  print("Tamanho do grupo 4 - 100% dos valores (Q4): ", len(grupos[3]))

  # Calcula a média e o desvio padrão dos dados
  mu, std = np.mean(valEnt), np.std(valEnt)
  # Histograma dos dados
  plt.hist(valEnt, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  p = norm.pdf(x, mu, std)
  # Plota a curva gaussiana
  plt.plot(x, p, 'k', linewidth=2, label="Gaussiana")
  # Configurações adicionais e exibição do gráfico
  plt.title('Data Histogram & Gaussian Curve')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()

def hipoteseNula(tuplasEntropia):
  print("\n")
  xEnt = [(ento[1]) for ento in (tuplasEntropia)]
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  #Shapiro-Wilk:
  statSW, pSW = stats.shapiro(xEnt)  # Obtém a estatística de teste e o valor-p
  validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
  print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
  #Kolmogorov-Smirnov
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
  validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
  print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
  #D'Agostino e Pearson's
  statAP, pAP = normaltest(xEnt)
  validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
  print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

  if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
    print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
  print("\n")

In [ ]:
train_dataset_mnist, test_dataset_mnist, user_groups_train_mnist, user_groups_test_mnist, all_images_np = pegarDadosIDDMNIST()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMM(all_images_np, base = 0, ftupla = 0)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

In [ ]:
train_dataset_mnist, test_dataset_mnist, user_groups_train_mnist, user_groups_test_mnist, all_images_np = pegarDadosIDDMNIST()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMM(all_images_np, base = 1, ftupla = 1)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

In [ ]:
train_dataset_mnist, test_dataset_mnist, user_groups_train_mnist, user_groups_test_mnist, all_images_np = pegarDadosIDDMNIST()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMM(all_images_np, base = 2, ftupla = 1)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

### Functions [NON-IDD DATASET]

In [ ]:
from mnist_noniid import get_dataset_mnist_extr_noniid
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import kstest, norm
from scipy import stats

def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def indicesEntropiaMedianMnist(train_X):
  train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))   # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #selecionando os indices de entropia entropia baixa abaixo da mediana.
  #indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] >= median])  #selecionando os indices de entropia entropia baixa abaixo da mediana.
  print('Valor da Mediana: ', median)
  train_X = np.array([train_X[i] for i in indices_filtrados]) # passa os indices selecionados para base de treino
  #tuplasEntropia = np.array([tuplasEntropia[i] for i in indices_filtrados]) # passa os indices selecionados para base de treino
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def mnistNONIDD(num_users_mnist = 400, nclass_mnist = 2, nsamples_mnist = 20, rate_unbalance_mnist = 1.0, p = 100, q = 5, plotar = 5):
  #Tratamento de erros
  num_users_mnist = 400 if num_users_mnist == 0 else num_users_mnist
  nclass_mnist = 2 if nclass_mnist == 0 else nclass_mnist
  nsamples_mnist = 20 if nsamples_mnist == 0 else  nsamples_mnist
  rate_unbalance_mnist = 1.0 if rate_unbalance_mnist == 0.0 else rate_unbalance_mnist
  p = 100 if p == 0 else p
  q = 5 if q == 0 else q
  plotar = 5 if plotar == 0 else plotar

  print("Dados da Importação: ")
  train_dataset_mnist, test_dataset_mnist, user_groups_train_mnist, user_groups_test_mnist = get_dataset_mnist_extr_noniid(num_users_mnist, nclass_mnist, nsamples_mnist, rate_unbalance_mnist)
  print("\n")
  print("Informações Gerais do Conjunto de Dados")
  print(f"Total de usuarios: {len(user_groups_train_mnist)}")

  if p == 1:
   for i in (range(int(len(user_groups_train_mnist)))):
     print(f"Grupo {i}: ", user_groups_train_mnist[i][:q], " ... ", user_groups_train_mnist[i][(nsamples_mnist - q):nsamples_mnist])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
  elif p >= 2:
    for i in (range(int(len(user_groups_train_mnist)/p))):
      print(f"Grupo {i}: ", user_groups_train_mnist[i][:q], " ... ", user_groups_train_mnist[i][(nsamples_mnist - q):nsamples_mnist])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
    print("\n")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("\n")
    g = (len(user_groups_train_mnist) - int(len(user_groups_train_mnist)/p))
    for g in range(g, len(user_groups_train_mnist)):
      print(f"Grupo {g}: ", user_groups_train_mnist[g][:q], " ... ", user_groups_train_mnist[g][(nsamples_mnist - q):nsamples_mnist])

  # Desnormalizando as imagens
  mean, std = 0.1307, 0.3081
  datasetGauss = []
  # Iterando para cada conjunto
  n = int(num_users_mnist/p) # qtd de imagens apartir da primeira
  print("\n")
  print(f"Calculando os valores para {n} Grupos: ")
  for user_idx in range(n):  # os primeiros `n` grupos
      all_images_np = [train_dataset_mnist[int(i)][0].numpy().squeeze() for i in user_groups_train_mnist[user_idx]]
      all_images_np = np.stack(all_images_np)

      #Removendo a normalização
      all_images_np = all_images_np * std + mean
      all_images_np = np.clip(all_images_np, 0, 1)  # Clipping intervalo [0,1]

      print(f"\nImagens do Grupo {user_idx+1}:")
      fig, axes = plt.subplots(1, plotar, figsize=(15,3))  # Criando 5 subplots lado a lado
      for idx, ax in enumerate(axes):
          image = all_images_np[idx]
          ax.imshow(image, cmap='gray')
          ax.set_title(f"I:{idx+1}")
          ax.axis('off')  # não mostrar os eixos
      plt.tight_layout()  # Ajusta o layout
      plt.show()

      #Calculando o valor da entropia para o Indice atual
      train_x, tuplasEntropia, entropiesLocal_ordenado = indicesEntropiaMedianMnist(all_images_np)
      e = 2  # imprimir os primeiros e últimos 5 elementos
      arr = train_x[0].flatten()
      output_str = "Valores do dataset Original (Chave|Valor): [" + ' '.join(map(str, arr[:e])) + " ... " + ' '.join(map(str, arr[-e:])) + "] ..."
      print(output_str)
      print("Valores do Conjunto (Chave|Valor) Não-Ordenado: ", tuplasEntropia[:q], " ...")
      print("Valores do Conjunto (Chave|Valor) Ordenado: ", entropiesLocal_ordenado[:q] , " ...")

      print("\n")
      valEntr = [(ento[1]) for ento in (tuplasEntropia)]
      xEnt = np.array(valEntr)
      #Shapiro-Wilk:
      statSW, pSW = stats.shapiro(xEnt)  # estatística de teste e o valor-p
      validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
      print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
      #Kolmogorov-Smirnov
      mu, std_dev = np.mean(xEnt), np.std(xEnt)
      statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
      validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
      print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
      #D'Agostino e Pearson's
      statAP, pAP = normaltest(xEnt)
      validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
      print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

      if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
        print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
        datasetGauss.append(train_x)
      print("\n")

      plt.hist(valEntr, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
      xmin, xmax = plt.xlim()
      mu, std_dev = np.mean(valEntr), np.std(valEntr)
      x = np.linspace(xmin, xmax, 100)
      y = norm.pdf(x, mu, std_dev)
      plt.plot(x, y, 'k-', lw=2, label='Curva Gaussiana')
      plt.title(f'Histograma dos Dados & Curva Gaussiana para o Usuário {user_idx}')
      plt.xlabel('Valor')
      plt.ylabel('Frequência')
      plt.legend()
      plt.show()

  print("Total de Distribuições Gaussianas Encontradas: ", len(datasetGauss), "| Total de Usuarios de até: ", len(user_groups_train_mnist))
  return datasetGauss

In [ ]:
datasetGauss = mnistNONIDD(num_users_mnist = 400, nclass_mnist = 2, nsamples_mnist = 20, rate_unbalance_mnist = 1.0, p = 1, q = 5, plotar = 5)

# Dataset [Cifar10]

## Functions [DADOS IDD]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy
from keras.src.utils.np_utils import to_categorical
from tensorflow.keras.datasets import cifar10
import time
from cifar10_noniid import get_dataset_cifar10_extr_noniid
from scipy.stats import norm
from scipy.stats import kstest
from scipy.stats import normaltest
from scipy import stats


def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def pegarDadosIDDCifar():
  num_users_cifar = 1
  nclass_cifar = 9
  nsamples_cifar = 1
  rate_unbalance_cifar = 0.0
  train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar = get_dataset_cifar10_extr_noniid(num_users_cifar, nclass_cifar, nsamples_cifar, rate_unbalance_cifar)
  print(train_dataset_cifar)
  all_images = [img[0].numpy().squeeze() for img in train_dataset_cifar]
  all_images_np = np.stack(all_images)
  #Removendo as mudanças da normaliação
  mean_rgb = np.array([0.4914, 0.4822, 0.4465])
  std_rgb = np.array([0.2023, 0.1994, 0.201])
  all_images_np = all_images_np * std_rgb[:, None, None] + mean_rgb[:, None, None]
  #all_images_np = np.transpose(all_images_np[0], (1, 2, 0))
  print(all_images_np.shape)
  # selecionando a primeira imagem
  image = all_images_np[0]
  # exibir a imagem
  image = image
  plt.show()
  all_labels = [lbl for _, lbl in train_dataset_cifar]
  plt.hist(all_labels, bins=10, edgecolor='black', alpha=0.7)
  plt.title('Class Distribution Histogram')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()
  return train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar, all_images_np

def iEMC(train_X, base = 0, ftupla = 0):
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  print('Valor da Mediana: ', median)
  if base == 1:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #indices de entropia entropia baixa abaixo da mediana.
  elif base == 2:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] >= median])  #indices de entropia entropia baixa abaixo da mediana.
  if base != 0:
    train_X = np.array([train_X[i] for i in indices_filtrados])
  if ftupla == 1:
    # passa os indices selecionados
    tuplasEntropia = ([tuplasEntropia[i] for i in indices_filtrados])
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado):
  print("Tamanho do Dataset: ", len(tuplasEntropia))
  print("Tamanho do divido pela Mediana: ", len(train_X))
  print("Entropia das imagens (Não-Ordenado | Original): ", tuplasEntropia[:5], " ...")
  print("Entropia das imagens (Ordenado | Valor): ", entropiesLocal_ordenado[:5], " ...")

  valEnt = [(ento[1]) for ento in (tuplasEntropia)]
  soma = sum(valEnt)
  print("Entropia Total Acumulada: ", soma)
  alvos = [0.25 * soma, 0.5 * soma, 0.75 * soma, soma]
  grupos = []
  grupo_atual = []
  soma_acumulada = 0
  index_alvo = 0
  for num in valEnt:
      grupo_atual.append(num)
      soma_acumulada += num
      if soma_acumulada >= alvos[index_alvo]:
          grupos.append(grupo_atual)
          grupo_atual = []
          index_alvo += 1
          if index_alvo == 4:
              break
  if grupo_atual:
      grupos.append(grupo_atual)

  print("Tamanho do Conjunto Analisado do Grafico a seguir: ", len(valEnt))
  print("Tamanho do grupo 1 - 25% dos valores (Q1): ", len(grupos[0]))
  print("Tamanho do grupo 2 - 50% dos valores (Q2): ", len(grupos[1]))
  print("Tamanho do grupo 3 - 75% dos valores (Q3): ", len(grupos[2]))
  print("Tamanho do grupo 4 - 100% dos valores (Q4): ", len(grupos[3]))

  # Calcula a média e o desvio padrão dos dados
  mu, std = np.mean(valEnt), np.std(valEnt)
  # Histograma dos dados
  plt.hist(valEnt, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  # Calcula a PDF da distribuição normal para cada valor de x
  p = norm.pdf(x, mu, std)
  # Plota a curva gaussiana
  plt.plot(x, p, 'k', linewidth=2, label="Gaussiana")
  plt.title('Data Histogram & Gaussian Curve')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()


def hipoteseNula(tuplasEntropia):
  print("\n")
  xEnt = [(ento[1]) for ento in (tuplasEntropia)]
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  #Shapiro-Wilk:
  statSW, pSW = stats.shapiro(xEnt)  # Obtém a estatística de teste e o valor-p
  validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
  print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
  #Kolmogorov-Smirnov
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
  validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
  print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
  #D'Agostino e Pearson's
  statAP, pAP = normaltest(xEnt)
  validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
  print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

  if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
    print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
  print("\n")

In [ ]:
train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar, all_images_np = pegarDadosIDDCifar()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMC(all_images_np, base = 0, ftupla = 0)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

In [ ]:
train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar, all_images_np = pegarDadosIDDCifar()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMC(all_images_np, base = 1, ftupla = 1)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

In [ ]:
train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar, all_images_np = pegarDadosIDDCifar()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMC(all_images_np, base = 2, ftupla = 1)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

## Cifar10 [Dados NON-IDD]

In [ ]:
from cifar10_noniid import get_dataset_cifar10_extr_noniid
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy import stats

def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def indicesEntropiaMedianCifar(train_X):
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #indices de entropia entropia baixa abaixo da mediana.
  #indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] >= median])  #indices de entropia entropia baixa abaixo da mediana.
  print('Valor da Mediana: ', median)
  train_X = np.array([train_X[i] for i in indices_filtrados]) # passa os indices selecionados
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def cifar10NONIDD(num_users_cifar = 400, nclass_cifar = 2, nsamples_cifar = 20, rate_unbalance_cifar = 1.0, p = 100, q = 5, plotar = 5):
  #Tratamento de erros
  num_users_cifar = 400 if num_users_cifar == 0 else num_users_cifar
  nclass_cifar = 2 if nclass_cifar == 0 else nclass_cifar
  nsamples_cifar = 20 if nsamples_cifar == 0 else  nsamples_cifar
  rate_unbalance_cifar = 1.0 if rate_unbalance_cifar == 0.0 else rate_unbalance_cifar
  p = 100 if p == 0 else p
  q = 5 if q == 0 else q
  plotar = 5 if plotar == 0 else plotar

  print("Dados da Importação: ")
  train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar = get_dataset_cifar10_extr_noniid(num_users_cifar, nclass_cifar, nsamples_cifar, rate_unbalance_cifar)
  print("\n")
  print("Informações Gerais do Conjunto de Dados")
  print(f"Total de usuarios: {len(user_groups_train_cifar)}")

  if p == 1:
   for i in (range(int(len(user_groups_train_cifar)))):
     print(f"Grupo {i}: ", user_groups_train_cifar[i][:q], " ... ", user_groups_train_cifar[i][(nsamples_cifar - q):nsamples_cifar])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
  elif p >= 2:
    for i in (range(int(len(user_groups_train_cifar)/p))):
      print(f"Grupo {i}: ", user_groups_train_cifar[i][:q], " ... ", user_groups_train_cifar[i][(nsamples_cifar - q):nsamples_cifar])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
    print("\n")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("\n")
    g = (len(user_groups_train_cifar) - int(len(user_groups_train_cifar)/p))
    for g in range(g, len(user_groups_train_cifar)):
      print(f"Grupo {g}: ", user_groups_train_cifar[g][:q], " ... ", user_groups_train_cifar[g][(nsamples_cifar - q):nsamples_cifar])

  # Desnormalizando as imagens
  mean_rgb = np.array([0.4914, 0.4822, 0.4465])
  std_rgb = np.array([0.2023, 0.1994, 0.201])

  # Iterando sobre cada conjunto de índices de usuário
  n = int(num_users_cifar/p) #qtd de imagens apartir da primeira
  print("\n")
  print(f"Calculando os valores para {n} Grupos: ")
  datasetGauss = []
  for user_idx in range(n):
      # Convertendo índices flutuantes para inteiros para obter as imagens usando esses índices
      all_images = [train_dataset_cifar[int(i)][0].numpy() for i in user_groups_train_cifar[user_idx]]
      all_images_np = np.stack(all_images)
      all_images_np = all_images_np * std_rgb[:, None, None] + mean_rgb[:, None, None]
      all_images_np = np.clip(all_images_np, 0, 1) # Clipping para garantir que a imagem esteja no intervalo [0,1]
      ##################################################################################
      print('\n')
      print('...... ......  ...... ......  ......  ......  ......  ...... ...... ...... ...... ...... ......  ...... ......  ......  ......  ......  ...... ...... ...... ......')
      print(f"Imagens do Grupo {user_idx}: ")
      fig, axes = plt.subplots(1, plotar, figsize=(15,3))  # Criando 5 subplots lado a lado
      for idx, ax in enumerate(axes):
          image = all_images_np[idx].transpose(1, 2, 0)
          ax.imshow(image)
          ax.set_title(f"Img {idx+1}")
          ax.axis('off')  # Para não mostrar os eixos
      plt.tight_layout()  # Ajusta o layout para evitar sobreposições
      plt.show()

      ##################################################################################
      #Calculando o valor da entropia para o Indice atual
      train_x, tuplasEntropia, entropiesLocal_ordenado = indicesEntropiaMedianCifar(all_images_np)
      e = 2  # para imprimir os primeiros e últimos 5 elementos
      arr = train_x[0].flatten()
      output_str = "Valores do dataset Original (Chave|Valor): [" + ' '.join(map(str, arr[:e])) + " ... " + ' '.join(map(str, arr[-e:])) + "] ..."
      print(output_str)
      print("Valores do Conjunto (Chave|Valor) Não-Ordenado: ", tuplasEntropia[:q], " ...")
      print("Valores do Conjunto (Chave|Valor) Ordenado: ", entropiesLocal_ordenado[:q] , " ...")


      print("\n")
      valEntr = [(ento[1]) for ento in (tuplasEntropia)]
      xEnt = np.array(valEntr)
      mu, std = np.mean(valEntr), np.std(valEntr)
      mu, std_dev = np.mean(xEnt), np.std(xEnt)
      #Shapiro-Wilk:
      statSW, pSW = stats.shapiro(xEnt)  # Obtém a estatística de teste e o valor-p
      validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
      print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
      #Kolmogorov-Smirnov
      mu, std_dev = np.mean(xEnt), np.std(xEnt)
      statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
      validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
      print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
      #D'Agostino e Pearson's
      statAP, pAP = normaltest(xEnt)
      validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
      print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

      if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
        print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
        datasetGauss.append(train_x)
      print("\n")


      plt.hist(valEntr, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
      xmin, xmax = plt.xlim()
      x = np.linspace(xmin, xmax, 100)
      p = norm.pdf(x, mu, std)
      plt.plot(x, p, 'k', linewidth=2, label="Gaussiana")
      plt.title(f'Histograma dos Dados & Curva Gaussiana para o Usuário {user_idx}')
      plt.xlabel('Valor')
      plt.ylabel('Frequência')
      plt.legend()
      plt.show()

  print("Total de Distribuições Gaussianas Encontradas: ", len(datasetGauss), "| Total de Usuarios de até: ", len(user_groups_train_cifar))
  return datasetGauss

In [ ]:
datasetGauss = cifar10NONIDD(num_users_cifar = 400, nclass_cifar = 2, nsamples_cifar = 20, rate_unbalance_cifar = 1.0, p = 1, q = 5, plotar = 5)

# Dataset [Cifar100]

## Cifar100 Functions [data IDD]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy
from keras.src.utils.np_utils import to_categorical
from tensorflow.keras.datasets import cifar100
import time
from cifar100_noniid import get_dataset_cifar100_extr_noniid
from scipy.stats import norm
from scipy.stats import kstest
from scipy.stats import normaltest
from scipy import stats


def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def pegarDadosIDDCifar100():
  num_users_cifar = 1
  nclass_cifar = 9
  nsamples_cifar = 1
  rate_unbalance_cifar = 0.0
  train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar = get_dataset_cifar100_extr_noniid(num_users_cifar, nclass_cifar, nsamples_cifar, rate_unbalance_cifar)
  print(train_dataset_cifar)
  all_images = [img[0].numpy().squeeze() for img in train_dataset_cifar]
  all_images_np = np.stack(all_images)
  #Removendo a mudança ao qual deixa os dados
  mean_rgb = np.array([0.5071, 0.4867, 0.4408])
  std_rgb = np.array([0.2675, 0.2565, 0.2761])

  all_images_np = all_images_np * std_rgb[:, None, None] + mean_rgb[:, None, None]
  print(all_images_np.shape)
  image = all_images_np[0]
  image = image
  plt.show()
  all_labels = [lbl for _, lbl in train_dataset_cifar]
  plt.hist(all_labels, bins=10, edgecolor='black', alpha=0.7)
  plt.title('Class Distribution Histogram')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()
  return train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar, all_images_np

def iEMC(train_X, base = 0, ftupla = 0):
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  print('Valor da Mediana: ', median)
  if base == 1:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #indices de entropia entropia baixa abaixo da mediana.
  elif base == 2:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] >= median])  #indices de entropia entropia baixa abaixo da mediana.
  if base != 0:
    train_X = np.array([train_X[i] for i in indices_filtrados]) # passa os indices selecionados
  if ftupla == 1:
    # passa os indices selecionados
    tuplasEntropia = ([tuplasEntropia[i] for i in indices_filtrados]) #
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado):
  print("Tamanho do Dataset: ", len(tuplasEntropia))
  print("Tamanho do divido pela Mediana: ", len(train_X))
  print("Entropia das imagens (Não-Ordenado | Original): ", tuplasEntropia[:5], " ...")
  print("Entropia das imagens (Ordenado | Valor): ", entropiesLocal_ordenado[:5], " ...")

  valEnt = [(ento[1]) for ento in (tuplasEntropia)]
  soma = sum(valEnt)
  print("Entropia Total Acumulada: ", soma)
  alvos = [0.25 * soma, 0.5 * soma, 0.75 * soma, soma]
  grupos = []
  grupo_atual = []
  soma_acumulada = 0
  index_alvo = 0
  for num in valEnt:
      grupo_atual.append(num)
      soma_acumulada += num
      if soma_acumulada >= alvos[index_alvo]:
          grupos.append(grupo_atual)
          grupo_atual = []
          index_alvo += 1
          if index_alvo == 4:
              break
  if grupo_atual:
      grupos.append(grupo_atual)

  print("Tamanho do Conjunto Analisado do Grafico a seguir: ", len(valEnt))
  print("Tamanho do grupo 1 - 25% dos valores (Q1): ", len(grupos[0]))
  print("Tamanho do grupo 2 - 50% dos valores (Q2): ", len(grupos[1]))
  print("Tamanho do grupo 3 - 75% dos valores (Q3): ", len(grupos[2]))
  print("Tamanho do grupo 4 - 100% dos valores (Q4): ", len(grupos[3]))

  # Calcula a média e o desvio padrão dos dados
  mu, std = np.mean(valEnt), np.std(valEnt)
  # Histograma dos dados
  plt.hist(valEnt, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  p = norm.pdf(x, mu, std)
  # Plota a curva gaussiana
  plt.plot(x, p, 'k', linewidth=2, label="Gaussiana")
  # Configurações adicionais e exibição do gráfico
  plt.title('Data Histogram & Gaussian Curve')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()


def hipoteseNula(tuplasEntropia):
  print("\n")
  xEnt = [(ento[1]) for ento in (tuplasEntropia)]
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  #Shapiro-Wilk:
  statSW, pSW = stats.shapiro(xEnt)  # Obtém a estatística de teste e o valor-p
  validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
  print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
  #Kolmogorov-Smirnov
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
  validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
  print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
  #D'Agostino e Pearson's
  statAP, pAP = normaltest(xEnt)
  validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
  print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

  if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
    print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
  print("\n")

In [ ]:
train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar, all_images_np = pegarDadosIDDCifar100()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMC(all_images_np, base = 0, ftupla = 0)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

## Cifar100 [data NON-IDD]

In [ ]:
from cifar100_noniid import get_dataset_cifar100_extr_noniid
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy import stats

def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def indicesEntropiaMedianCifar(train_X):
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #indices de entropia entropia baixa abaixo da mediana.
  print('Valor da Mediana: ', median)
  train_X = np.array([train_X[i] for i in indices_filtrados]) # passa os indices selecionados
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def cifar100NONIDD(num_users_cifar = 400, nclass_cifar = 2, nsamples_cifar = 20, rate_unbalance_cifar = 1.0, p = 100, q = 5, plotar = 5):
  #Tratamento de erros
  num_users_cifar = 400 if num_users_cifar == 0 else num_users_cifar
  nclass_cifar = 2 if nclass_cifar == 0 else nclass_cifar
  nsamples_cifar = 20 if nsamples_cifar == 0 else  nsamples_cifar
  rate_unbalance_cifar = 1.0 if rate_unbalance_cifar == 0.0 else rate_unbalance_cifar
  p = 100 if p == 0 else p
  q = 5 if q == 0 else q
  plotar = 5 if plotar == 0 else plotar

  print("Dados da Importação: ")
  train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar = get_dataset_cifar100_extr_noniid(num_users_cifar, nclass_cifar, nsamples_cifar, rate_unbalance_cifar)
  print("\n")
  print("Informações Gerais do Conjunto de Dados")
  print(f"Total de usuarios: {len(user_groups_train_cifar)}")

  if p == 1:
   for i in (range(int(len(user_groups_train_cifar)))):
     print(f"Grupo {i}: ", user_groups_train_cifar[i][:q], " ... ", user_groups_train_cifar[i][(nsamples_cifar - q):nsamples_cifar])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
  elif p >= 2:
    for i in (range(int(len(user_groups_train_cifar)/p))):
      print(f"Grupo {i}: ", user_groups_train_cifar[i][:q], " ... ", user_groups_train_cifar[i][(nsamples_cifar - q):nsamples_cifar])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
    print("\n")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("\n")
    g = (len(user_groups_train_cifar) - int(len(user_groups_train_cifar)/p))
    for g in range(g, len(user_groups_train_cifar)):
      print(f"Grupo {g}: ", user_groups_train_cifar[g][:q], " ... ", user_groups_train_cifar[g][(nsamples_cifar - q):nsamples_cifar])

  # Desnormalizando as imagens
  mean_rgb = np.array([0.5071, 0.4867, 0.4408])
  std_rgb = np.array([0.2675, 0.2565, 0.2761])

  # Iterando sobre cada conjunto de índices de usuário
  n = int(num_users_cifar/p) #qtd de imagens apartir da primeira
  print("\n")
  print(f"Calculando os valores para {n} Grupos: ")
  datasetGauss = []
  for user_idx in range(n):
      # Convertendo índices flutuantes para inteiros e obtemos as imagens usando esses índices
      all_images = [train_dataset_cifar[int(i)][0].numpy() for i in user_groups_train_cifar[user_idx]]
      all_images_np = np.stack(all_images)
      all_images_np = all_images_np * std_rgb[:, None, None] + mean_rgb[:, None, None]
      all_images_np = np.clip(all_images_np, 0, 1) # Clipping para garantir que a imagem esteja no intervalo [0,1]
      ##################################################################################
      print('\n')
      print('...... ......  ...... ......  ......  ......  ......  ...... ...... ...... ...... ...... ......  ...... ......  ......  ......  ......  ...... ...... ...... ......')
      print(f"Imagens do Grupo {user_idx}: ")
      fig, axes = plt.subplots(1, plotar, figsize=(15,3))  # Criando 5 subplots lado a lado
      for idx, ax in enumerate(axes):
          image = all_images_np[idx].transpose(1, 2, 0)
          ax.imshow(image)
          ax.set_title(f"Img {idx+1}")
          ax.axis('off')  # Para não mostrar os eixos
      plt.tight_layout()  # Ajusta o layout para evitar sobreposições
      plt.show()

      ##################################################################################
      #Calculando o valor da entropia para o Indice atual
      train_x, tuplasEntropia, entropiesLocal_ordenado = indicesEntropiaMedianCifar(all_images_np)
      e = 2  # para imprimir os primeiros e últimos 5 elementos
      arr = train_x[0].flatten()
      output_str = "Valores do dataset Original (Chave|Valor): [" + ' '.join(map(str, arr[:e])) + " ... " + ' '.join(map(str, arr[-e:])) + "] ..."
      print(output_str)
      print("Valores do Conjunto (Chave|Valor) Não-Ordenado: ", tuplasEntropia[:q], " ...")
      print("Valores do Conjunto (Chave|Valor) Ordenado: ", entropiesLocal_ordenado[:q] , " ...")


      print("\n")
      valEntr = [(ento[1]) for ento in (tuplasEntropia)]
      xEnt = np.array(valEntr)
      mu, std = np.mean(valEntr), np.std(valEntr)
      mu, std_dev = np.mean(xEnt), np.std(xEnt)
      #Shapiro-Wilk:
      statSW, pSW = stats.shapiro(xEnt)  # Obtém a estatística de teste e o valor-p
      validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
      print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
      #Kolmogorov-Smirnov
      mu, std_dev = np.mean(xEnt), np.std(xEnt)
      statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
      validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
      print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
      #D'Agostino e Pearson's
      statAP, pAP = normaltest(xEnt)
      validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
      print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

      if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
        print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
        datasetGauss.append(train_x)
      print("\n")


      plt.hist(valEntr, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
      xmin, xmax = plt.xlim()
      x = np.linspace(xmin, xmax, 100)
      p = norm.pdf(x, mu, std)
      plt.plot(x, p, 'k', linewidth=2, label="Gaussiana")
      plt.title(f'Histograma dos Dados & Curva Gaussiana para o Usuário {user_idx}')
      plt.xlabel('Valor')
      plt.ylabel('Frequência')
      plt.legend()
      plt.show()

  print("Total de Distribuições Gaussianas Encontradas: ", len(datasetGauss), "| Total de Usuarios de até: ", len(user_groups_train_cifar))
  return datasetGauss

In [ ]:
datasetGauss = cifar100NONIDD(num_users_cifar = 400, nclass_cifar = 3, nsamples_cifar = 20, rate_unbalance_cifar = 1.0, p = 1, q = 5, plotar = 5)

# Dataset Fashion Mnist

### Funções [IDD DATASET]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy
from keras.src.utils.np_utils import to_categorical
from tensorflow.keras.datasets import cifar10
import time
from fashionmnist_noniid import get_dataset_fashion_mnist_extr_noniid
from scipy.stats import norm
from scipy.stats import kstest
from scipy.stats import normaltest
from scipy import stats


def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def pegarDadosIDDFashion():
  num_users_fashion = 1
  nclass_fashion = 9
  nsamples_fashion = 1
  rate_unbalance_fashion = 0.0
  train_dataset_fashion, test_dataset_fashion, user_groups_train_fashion, user_groups_test_fashion = get_dataset_fashion_mnist_extr_noniid(num_users_fashion, nclass_fashion, nsamples_fashion, rate_unbalance_fashion)
  print(train_dataset_fashion)
  all_images = [img[0].numpy().squeeze() for img in train_dataset_fashion]
  all_images_np = np.stack(all_images)
  #Removendo a normalização
  mean, std = 0.2860, 0.3530
  all_images_np = all_images_np * std + mean
  print(all_images_np.shape)
  plt.show()
  all_labels = [lbl for _, lbl in train_dataset_fashion]
  plt.hist(all_labels, bins=10, edgecolor='black', alpha=0.7)
  plt.title('Class Distribution Histogram')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()
  return train_dataset_fashion, test_dataset_fashion, user_groups_train_fashion, user_groups_test_fashion, all_images_np

def iEMC(train_X, base = 0, ftupla = 0):
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  print('Valor da Mediana: ', median)
  if base == 1:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #indices de entropia entropia baixa abaixo da mediana.
  elif base == 2:
    indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] >= median])  #indices de entropia entropia baixa abaixo da mediana.
  if base != 0:
    train_X = np.array([train_X[i] for i in indices_filtrados]) # passa os indices selecionados para base de treino
  if ftupla == 1:
    tuplasEntropia = ([tuplasEntropia[i] for i in indices_filtrados]) # passa os indices selecionados
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado):
  print("Tamanho do Dataset: ", len(tuplasEntropia))
  print("Tamanho do divido pela Mediana: ", len(train_X))
  print("Entropia das imagens (Não-Ordenado | Original): ", tuplasEntropia[:5], " ...")
  print("Entropia das imagens (Ordenado | Valor): ", entropiesLocal_ordenado[:5], " ...")

  valEnt = [(ento[1]) for ento in (tuplasEntropia)]
  soma = sum(valEnt)
  print("Entropia Total Acumulada: ", soma)
  alvos = [0.25 * soma, 0.5 * soma, 0.75 * soma, soma]
  grupos = []
  grupo_atual = []
  soma_acumulada = 0
  index_alvo = 0
  for num in valEnt:
      grupo_atual.append(num)
      soma_acumulada += num
      if soma_acumulada >= alvos[index_alvo]:
          grupos.append(grupo_atual)
          grupo_atual = []
          index_alvo += 1
          if index_alvo == 4:
              break
  if grupo_atual:
      grupos.append(grupo_atual)

  print("Tamanho do Conjunto Analisado do Grafico a seguir: ", len(valEnt))
  print("Tamanho do grupo 1 - 25% dos valores (Q1): ", len(grupos[0]))
  print("Tamanho do grupo 2 - 50% dos valores (Q2): ", len(grupos[1]))
  print("Tamanho do grupo 3 - 75% dos valores (Q3): ", len(grupos[2]))
  print("Tamanho do grupo 4 - 100% dos valores (Q4): ", len(grupos[3]))

  # Calcula a média e o desvio padrão dos dados
  mu, std = np.mean(valEnt), np.std(valEnt)
  # Histograma dos dados
  plt.hist(valEnt, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  p = norm.pdf(x, mu, std)
  # Plota a curva gaussiana
  plt.plot(x, p, 'k', linewidth=2, label="Gaussiana")
  # Configurações adicionais e exibição do gráfico
  plt.title('Data Histogram & Gaussian Curve')
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()


def hipoteseNula(tuplasEntropia):
  print("\n")
  xEnt = [(ento[1]) for ento in (tuplasEntropia)]
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  #Shapiro-Wilk:
  statSW, pSW = stats.shapiro(xEnt)  # Obtém a estatística de teste e o valor-p
  validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
  print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
  #Kolmogorov-Smirnov
  mu, std_dev = np.mean(xEnt), np.std(xEnt)
  statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
  validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
  print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
  #D'Agostino e Pearson's
  statAP, pAP = normaltest(xEnt)
  validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
  print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

  if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
    (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
    print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
  print("\n")

In [ ]:
train_dataset_cifar, test_dataset_cifar, user_groups_train_cifar, user_groups_test_cifar, all_images_np = pegarDadosIDDFashion()
#base = 0, ftupla = 0 : distribuição dos dados completos
#base = 1, ftupla = 1 : distribuição dos dados a esquerda da mediana
#base = 2, ftupla = 1 : distribuição dos dados a direita da mediana
train_X, tuplasEntropia, entropiesLocal_ordenado= iEMC(all_images_np, base = 0, ftupla = 0)
hipoteseNula(tuplasEntropia)
entropiaEstatisticas(train_X, tuplasEntropia, entropiesLocal_ordenado)

### Funções [NON-IDD DATASET]

In [ ]:
from fashionmnist_noniid import get_dataset_fashion_mnist_extr_noniid
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import kstest, norm
from scipy import stats
from scipy.stats import normaltest

def entropiaBR(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    result = round(result, 17)
    return result

def indicesEntropiaMedianFashionMnist(train_X):
  train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))   # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  tuplasEntropia = [(index, entropiaBR(img.flatten())) for index, img in enumerate(train_X)]
  entropiesLocal_ordenado = sorted(tuplasEntropia, key=lambda x: x[1]) #ordenação com base na entropia
  n = len(entropiesLocal_ordenado)
  if n % 2 == 1:
      median = entropiesLocal_ordenado[n // 2][1] # mediana elemento do meio - impares
  else:
      median = (entropiesLocal_ordenado[n // 2 - 1][1] + entropiesLocal_ordenado[n // 2][1]) / 2.0  # mediana elemento do meio-pares
  indices_filtrados = np.array([item[0] for item in entropiesLocal_ordenado if item[1] <= median])  #indices de entropia entropia baixa abaixo da mediana.
  print('Valor da Mediana: ', median)
  train_X = np.array([train_X[i] for i in indices_filtrados]) # passa os indices selecionados para base de treino
  return train_X, tuplasEntropia, entropiesLocal_ordenado

def fashionMnistNONIDD(num_users_fashion_mnist = 400, nclass_fashion_mnist = 2, nsamples_fashion_mnist = 20, rate_unbalance_fashion_mnist = 1.0, p = 100, q = 5, plotar = 5):
  #Tratamento de erros
  num_users_fashion_mnist = 400 if num_users_fashion_mnist == 0 else num_users_fashion_mnist
  nclass_fashion_mnist = 2 if nclass_fashion_mnist == 0 else nclass_fashion_mnist
  nsamples_fashion_mnist = 20 if nsamples_fashion_mnist == 0 else  nsamples_fashion_mnist
  rate_unbalance_fashion_mnist = 1.0 if rate_unbalance_fashion_mnist == 0.0 else rate_unbalance_fashion_mnist
  p = 100 if p == 0 else p
  q = 5 if q == 0 else q
  plotar = 5 if plotar == 0 else plotar

  print("Dados da Importação: ")
  train_dataset_fashion_mnist, test_dataset_fashion_mnist, user_groups_train_fashion_mnist, user_groups_test_fashion_mnist = get_dataset_fashion_mnist_extr_noniid(num_users_fashion_mnist, nclass_fashion_mnist, nsamples_fashion_mnist, rate_unbalance_fashion_mnist)
  print("\n")
  print("Informações Gerais do Conjunto de Dados")
  print(f"Total de usuarios: {len(user_groups_train_fashion_mnist)}")

  if p == 1:
   for i in (range(int(len(user_groups_train_fashion_mnist)))):
     print(f"Grupo {i}: ", user_groups_train_fashion_mnist[i][:q], " ... ", user_groups_train_fashion_mnist[i][(nsamples_fashion_mnist - q):nsamples_fashion_mnist])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
  elif p >= 2:
    for i in (range(int(len(user_groups_train_fashion_mnist)/p))):
      print(f"Grupo {i}: ", user_groups_train_fashion_mnist[i][:q], " ... ", user_groups_train_fashion_mnist[i][(nsamples_fashion_mnist - q):nsamples_fashion_mnist])   #imprimindo os 5 primeiros e os 5 ultimos de cada grupo
    print("\n")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("Grupo n:  [...... ......  ...... ......  ......]  ...  [......  ...... ...... ...... ......]")
    print("\n")
    g = (len(user_groups_train_fashion_mnist) - int(len(user_groups_train_fashion_mnist)/p))
    for g in range(g, len(user_groups_train_fashion_mnist)):
      print(f"Grupo {g}: ", user_groups_train_fashion_mnist[g][:q], " ... ", user_groups_train_fashion_mnist[g][(nsamples_fashion_mnist - q):nsamples_fashion_mnist])

  # Desnormalizando as imagens
  mean, std = 0.2860, 0.3530
  datasetGauss = []
  # Iterando sobre cada conjunto
  n = int(num_users_fashion_mnist/p) # qtd de imagens apartir da primeira
  print("\n")
  print(f"Calculando os valores para {n} Grupos: ")
  for user_idx in range(n):  # itera sobre os primeiros `n` grupos
      all_images_np = [train_dataset_fashion_mnist[int(i)][0].numpy().squeeze() for i in user_groups_train_fashion_mnist[user_idx]]
      all_images_np = np.stack(all_images_np)

      #Removendo a mudança ao qual deixa os dados
      all_images_np = all_images_np * std + mean
      all_images_np = np.clip(all_images_np, 0, 1)  # Clipping no intervalo [0,1]

      print(f"\nImagens do Grupo {user_idx+1}:")
      fig, axes = plt.subplots(1, plotar, figsize=(15,3))  # Criando 5 subplots lado a lado
      for idx, ax in enumerate(axes):
          image = all_images_np[idx]
          ax.imshow(image, cmap='gray')
          ax.set_title(f"I:{idx+1}")
          ax.axis('off')
      plt.tight_layout()
      plt.show()

      #Calculando o valor da entropia para o Indice atual
      train_x, tuplasEntropia, entropiesLocal_ordenado = indicesEntropiaMedianFashionMnist(all_images_np)
      e = 2  # para imprimir os primeiros e últimos 5 elementos
      arr = train_x[0].flatten()
      output_str = "Valores do dataset Original (Chave|Valor): [" + ' '.join(map(str, arr[:e])) + " ... " + ' '.join(map(str, arr[-e:])) + "] ..."
      print(output_str)
      print("Valores do Conjunto (Chave|Valor) Não-Ordenado: ", tuplasEntropia[:q], " ...")
      print("Valores do Conjunto (Chave|Valor) Ordenado: ", entropiesLocal_ordenado[:q] , " ...")

      print("\n")
      valEntr = [(ento[1]) for ento in (tuplasEntropia)]
      xEnt = np.array(valEntr)
      #Shapiro-Wilk:
      statSW, pSW = stats.shapiro(xEnt)  # Obtém a estatística de teste e o valor-p
      validacaoSW = "Não Rejeitamos H0" if pSW > 0.05 else "Rejeitamos H0"
      print("Cal. Shapiro-Wilk, Statistica: %.13f" % statSW, "Valor-p: %.10f" % pSW, "Validação de Hipotese H0:", validacaoSW)
      #Kolmogorov-Smirnov
      mu, std_dev = np.mean(xEnt), np.std(xEnt)
      statKS, pKS = kstest(xEnt, 'norm', args=(mu, std_dev))
      validacaoKS = "Não Rejeitamos H0" if pKS > 0.05 else "Rejeitamos H0"
      print("Cal. Kolmogorov-Smirnov, Statistica: %.10f" % statKS, "Valor-p: %.10f" % pKS, "Validação de Hipotese H0:", validacaoKS)
      #D'Agostino e Pearson's
      statAP, pAP = normaltest(xEnt)
      validacaoAP = "Não Rejeitamos H0" if pAP > 0.05 else "Rejeitamos H0"
      print("Cal. D'Agostino e Pearson's, Statistica: %.10f" % statAP, "Valor-p: %.10f" % pAP, "Validação de Hipotese H0:", validacaoAP)

      if (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0") or \
        (validacaoSW == "Não Rejeitamos H0" and validacaoKS == "Não Rejeitamos H0" and validacaoAP == "Não Rejeitamos H0"):
        print("Com base em uma validação dupla identificamos de que não há evidência suficiente para rejeitar a hipótese nula. A amostra parece seguir uma distribuição normal.")
        datasetGauss.append(train_x)
      print("\n")

      plt.hist(valEntr, bins=10, edgecolor='black', alpha=0.7, density=True, label="Dados")
      xmin, xmax = plt.xlim()
      mu, std_dev = np.mean(valEntr), np.std(valEntr)
      x = np.linspace(xmin, xmax, 100)
      y = norm.pdf(x, mu, std_dev)
      plt.plot(x, y, 'k-', lw=2, label='Curva Gaussiana')
      plt.title(f'Histograma dos Dados & Curva Gaussiana para o Usuário {user_idx}')
      plt.xlabel('Valor')
      plt.ylabel('Frequência')
      plt.legend()
      plt.show()

  print("Total de Distribuições Gaussianas Encontradas: ", len(datasetGauss), "| Total de Usuarios de até: ", len(user_groups_train_fashion_mnist))
  return datasetGauss

In [ ]:
datasetGauss = fashionMnistNONIDD(num_users_fashion_mnist = 400, nclass_fashion_mnist = 2, nsamples_fashion_mnist = 20, rate_unbalance_fashion_mnist = 1.0, p = 1, q = 5, plotar = 5)